# AI-Driven Early Prediction of Pulmonary Fibrosis Using Deep Learning


In [13]:
import os
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. CONFIGURATION
# ==========================================
CONFIG = {
    "lr": 2e-3,
    "weight_decay": 1e-4,
    "batch_size": 128,
    "epochs": 200,
    "n_folds": 5,
    "quantiles": [0.2, 0.5, 0.8], 
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "data_dir": "../input/osic-pulmonary-fibrosis-progression",
    # Point to your extracted features file
    "biomarker_path": "../input/feature-extraction-u-net-segmentation/master_dataset.csv"
}

# ==========================================
# 2. DATA PREPROCESSING (ROBUST MERGE)
# ==========================================
def preprocess_data(config):
    global TARGET_SCALER
    
    # 1. Load Clinical Data (Guarantees Sex, Age, Weeks exist)
    clinical_df = pd.read_csv(f"{config['data_dir']}/train.csv")
    
    # 2. Load Biomarkers
    biomarkers_df = pd.read_csv(config['biomarker_path'])
    
    # 3. Robust Merge Logic
    # We only want the image features from the biomarkers file to avoid duplicates
    image_features = [
        'lung_vol_ml', 'hu_mean', 'hu_std', 'hu_skew', 'hu_kurt',
        'glcm_contrast', 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation'
    ]
    
    # Filter biomarkers_df to only keep Patient + Image Features
    # This prevents 'Sex_x', 'Sex_y' conflicts if columns repeat
    cols_to_keep = ['Patient'] + [c for c in image_features if c in biomarkers_df.columns]
    
    # Remove duplicates in biomarkers (keep 1 row per patient)
    biomarkers_clean = biomarkers_df[cols_to_keep].drop_duplicates(subset=['Patient'])
    
    # MERGE: Clinical (Left) + Biomarkers (Right)
    train = clinical_df.merge(biomarkers_clean, on='Patient', how='inner')
    
    # 4. Feature Engineering (Baseline & Relative Weeks)
    train['Weeks'] = train['Weeks'].astype(int)
    train.sort_values(['Patient', 'Weeks'], inplace=True)
    
    baseline = train.groupby('Patient').first().reset_index()
    baseline = baseline[['Patient', 'FVC', 'Percent']].rename(columns={'FVC': 'Base_FVC', 'Percent': 'Base_Percent'})
    train = train.merge(baseline, on='Patient', how='left')
    
    base_weeks = train.groupby('Patient')['Weeks'].min().reset_index().rename(columns={'Weeks': 'Base_Week'})
    train = train.merge(base_weeks, on='Patient', how='left')
    train['Relative_Weeks'] = train['Weeks'] - train['Base_Week']
    
    # 5. Feature Scaling
    scaler = RobustScaler()
    # Dynamic check for available columns
    available_img_feats = [c for c in image_features if c in train.columns]
    
    num_cols = ['Age', 'Base_FVC', 'Base_Percent', 'Relative_Weeks'] + available_img_feats
    train[num_cols] = scaler.fit_transform(train[num_cols])
    
    # 6. Encoding (Now guaranteed to work)
    train['Sex'] = train['Sex'].apply(lambda x: 1 if x == 'Male' else 0)
    train['Smk_Ex'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Ex-smoker' else 0)
    train['Smk_Cur'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Currently smokes' else 0)
    
    feature_cols = num_cols + ['Sex', 'Smk_Ex', 'Smk_Cur']
    
    # 7. Target Scaling
    TARGET_SCALER.fit(train[['FVC']])
    train['FVC_scaled'] = TARGET_SCALER.transform(train[['FVC']])
    
    print(f"✅ Preprocessing Complete. Final Shape: {train.shape}")
    print(f"✅ Features Used: {len(feature_cols)}")
    
    return train, feature_cols

# ==========================================
# 3. MODEL: QUANTILE MLP
# ==========================================
class QuantileMLP(nn.Module):
    def __init__(self, input_dim, quantiles):
        super().__init__()
        hidden = 128
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden), nn.BatchNorm1d(hidden), nn.LeakyReLU(0.1), nn.Dropout(0.3),
            nn.Linear(hidden, hidden), nn.BatchNorm1d(hidden), nn.LeakyReLU(0.1), nn.Dropout(0.3),
            nn.Linear(hidden, len(quantiles))
        )
    def forward(self, x): return self.net(x)

def quantile_loss(preds, target, quantiles):
    """Pinball Loss on SCALED targets."""
    assert not target.requires_grad
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i].unsqueeze(1)
        loss = torch.max((q-1) * errors, q * errors)
        losses.append(loss)
    return torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))

# ==========================================
# 4. METRIC CALCULATION
# ==========================================
def calculate_metrics(y_true, q_preds_real):
    """
    Calculates metrics on UNSCALED (Real) values.
    """
    q20 = q_preds_real[:, 0]
    q50 = q_preds_real[:, 1]
    q80 = q_preds_real[:, 2]
    
    # Sigma (Uncertainty)
    sigma = q80 - q20
    sigma_clipped = np.maximum(sigma, 70)
    
    # LLL
    delta = np.minimum(np.abs(y_true - q50), 1000)
    lll = - (np.sqrt(2) * delta / sigma_clipped) - np.log(np.sqrt(2) * sigma_clipped)
    
    # Regression Metrics
    mse = mean_squared_error(y_true, q50)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, q50)
    r2 = r2_score(y_true, q50)
    rmae = mae / (np.mean(np.abs(y_true)) + 1e-6)
    
    return np.mean(lll), mse, rmse, mae, rmae, r2

# ==========================================
# 5. TRAINING LOOP
# ==========================================
def train_model():
    df, features = preprocess_data(CONFIG)
    patients = df['Patient'].unique()
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=42)
    
    global_trues, global_preds = [], []
    
    print(f"🚀 Training Direct FVC Quantile Regression on {len(df)} visits.")
    print(f"Features: {features}")
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(patients)):
        train_p, val_p = patients[train_idx], patients[val_idx]
        train_data = df[df['Patient'].isin(train_p)]
        val_data = df[df['Patient'].isin(val_p)]
        
        # Train on SCALED FVC
        X_train = torch.tensor(train_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_train_scaled = torch.tensor(train_data['FVC_scaled'].values, dtype=torch.float32).unsqueeze(1).to(CONFIG['device'])
        
        X_val = torch.tensor(val_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        
        # Validation on RAW FVC (for final metrics)
        y_val_real = val_data['FVC'].values
        
        model = QuantileMLP(len(features), CONFIG['quantiles']).to(CONFIG['device'])
        optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['epochs'], eta_min=1e-6)
        
        best_lll = -float('inf')
        best_preds_real = None
        
        for epoch in range(CONFIG['epochs']):
            model.train()
            optimizer.zero_grad()
            preds_scaled = model(X_train)
            
            # Loss on SCALED targets
            loss = quantile_loss(preds_scaled, y_train_scaled, CONFIG['quantiles'])
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            # Validation
            model.eval()
            with torch.no_grad():
                val_preds_scaled = model(X_val)
                
                # UNSCALE Predictions for metrics
                val_preds_real = TARGET_SCALER.inverse_transform(val_preds_scaled.cpu().numpy())
                
                lll, _, _, _, _, _ = calculate_metrics(y_val_real, val_preds_real)
                
            if lll > best_lll:
                best_lll = lll
                best_preds_real = val_preds_real
                
        print(f"Fold {fold+1} Best | LLL: {best_lll:.4f}")
        
        global_trues.extend(y_val_real)
        global_preds.extend(best_preds_real)

    # --- FINAL SCORE ---
    g_true = np.array(global_trues)
    g_pred = np.array(global_preds) 
    
    lll, mse, rmse, mae, rmae, r2 = calculate_metrics(g_true, g_pred)
    
    print("\n" + "="*50)
    print("🏆 FINAL DIRECT FVC QUANTILE RESULTS")
    print("="*50)
    print(f"R² (FVC Correlation) : {r2:.4f}   (Target > 0.88)")
    print(f"MSE (mL²)            : {mse:.4f}")
    print(f"RMSE (mL)            : {rmse:.4f}  (Target < 170)")
    print(f"MAE (mL)             : {mae:.4f}")
    print(f"RMAE (Relative Error): {rmae:.4f}")
    print(f"LLL (OSIC Metric)    : {lll:.4f}   (Target > -6.64)")
    print("="*50)

if __name__ == "__main__":
    train_model()

✅ Preprocessing Complete. Final Shape: (1549, 23)
✅ Features Used: 16
🚀 Training Direct FVC Quantile Regression on 1549 visits.
Features: ['Age', 'Base_FVC', 'Base_Percent', 'Relative_Weeks', 'lung_vol_ml', 'hu_mean', 'hu_std', 'hu_skew', 'hu_kurt', 'glcm_contrast', 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation', 'Sex', 'Smk_Ex', 'Smk_Cur']
Fold 1 Best | LLL: -6.8408
Fold 2 Best | LLL: -6.9963
Fold 3 Best | LLL: -6.8772
Fold 4 Best | LLL: -6.9388
Fold 5 Best | LLL: -6.8931

🏆 FINAL DIRECT FVC QUANTILE RESULTS
R² (FVC Correlation) : 0.9049   (Target > 0.88)
MSE (mL²)            : 65878.0976
RMSE (mL)            : 256.6673  (Target < 170)
MAE (mL)             : 185.8179
RMAE (Relative Error): 0.0691
LLL (OSIC Metric)    : -6.9088   (Target > -6.64)


## Hurray